In [1]:
import os
import random
import re
import numpy as np
import shutil
import pandas as pd

In [2]:
rng = np.random.default_rng(seed=152) # old number=42, 

In [ ]:
data_folder = "output_19_01_2025/"
folder_path_outputs = '/app/nse/outputs/' + data_folder
file_path_results = '/app/nse/results/' + data_folder

In [ ]:
def read_data(filename):
    with open(filename, 'r') as file:
        # Skip the first three lines
        for _ in range(3):
            next(file)
        
        # Read the rest of the file
        data = np.loadtxt(file)
    
    return data

In [ ]:
def get_folder_paths(folder_path, folder_pattern):
    folder_list = [folder_path + f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f)) and re.search(folder_pattern, f)]
    folder_list.sort(key=os.path.getctime)
    return folder_list

In [ ]:
def extract_number(filename):
    match = re.search(r'C\[(\d+)\]-avg-\.plt', filename)
    return int(match.group(1)) if match else float('inf') 

In [ ]:
def sort_filenames(filenames):
    return sorted(filenames, key=extract_number)

In [ ]:
def get_filenames(file_path, file_pattern):
    file_list = [f for f in os.listdir(file_path) if os.path.isfile(os.path.join(file_path, f)) and re.search(file_pattern, f)]
    file_list = sort_filenames(file_list)
    return file_list

In [3]:
def add_text_to_file(file_path, line_number, text_to_add):
    # Read the contents of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Insert the new text at the specified line number
    lines.insert(line_number - 1, text_to_add + '\n')

    # Write the modified contents back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

In [4]:
def change_lines(file_path, lines_to_change, changed_lines):
    # Read the contents of the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    i = 0
    for idx, line in enumerate(lines):
        if re.search(lines_to_change[i], line):
            lines[idx] = changed_lines[i]
            i+=1
            if i == len(changed_lines):
                break
    if len(changed_lines) != i:
        print(len(changed_lines) - i, "lines not changed")
     
    # Write the modified contents back to the file
    with open(file_path, 'w') as file:
        file.writelines(lines)

In [ ]:
outputs_folder_names = get_folder_paths(folder_path_outputs, "output*")

In [ ]:
for folder in outputs_folder_names:
    config_path = folder + "/config.txt"
    data_config = read_data(config_path)
    

In [10]:

for idx in range(n_files):
	
	file_path = shutil.copyfile(file_path_initial, file_path_output + "config-plume-ex_" + str(idx) + ".txt")
	
	u = rng.uniform(u_min, u_max)
	value = rng.uniform(value_min, value_max)
	z0_m = rng.uniform(z0_min, z0_max)
	T0 = rng.uniform(T0_min, T0_max)
	gradz = rng.uniform(gradz_min, gradz_max)

	lines_changed = [
		f'	U = {u}; V = 0.0;			# [m/s] \n',
		f'		value = {value};	# sensible heat flux [K*m/s]\n',
		f'	z0_m = {z0_m};		# aerodynamic roughness [m]\n',
		f'		surface_value = {T0};	# initial boundary layer temperature [K]\n',
		f'		grad_z = {gradz};		# temperature gradient above boundary layer [K/m]\n',
		f'	num = {N};	# number of tracers, skipped if not defined'
	]
	for i in range(N, 0, -1):
		p = rng.uniform(p_min, p_max)
		y = rng.uniform(y_min, y_max)
		z = rng.triangular(z_min, z_mode, z_max)
		
		y_list.append(y)
		z_list.append(z)
		u_list.append(u)
		value_list.append(value)
		z0_list.append(z0_m)
		T0_list.append(T0)
		gradz_list.append(gradz)
		p_list.append(p)

		# Define the text to add
		text_to_add = f"""
	tracer_{i} {{ 
		diffusivity = phys.xi;

		surface {{ 
			flux = 0.0;
		}}

		# --- point emission source [optional]
		point_emission {{
			type = "gaussian";		# "gaussian" || "box"

			# --- source intensity [ppb * m^3 / s]
			value = {p} * (2.0 / 3.14) * (1.0 / 2.46) * 1000.0 * 1000.0 * 100.0;

			# --- active in [begin, end], [time.begin, time.end] if not defined
			begin = 0.5 * 3600.0;	# [s]

			xpos = domain.x + 0.25 * domain.length;		# [m]
			ypos = domain.y + {y};		# [m]
			zpos = {z};		# [m]
			
			sx = 50.0;		# [m]
			sy = 50.0;		# [m]
			sz = 25.0;		# [m]

			# --- OR set 'box'
			# type = "box";
			
			# xmin = 450.0; xmax = 550.0;	# [m]
			# ymin = 450.0; ymax = 550.0;	# [m]
			# zmin = 50.0; zmax = 250.0;	# [m]
		}}

		# --- boundary conditions [optional]
		boundary_conditions
		{{
			# --- default: -xy periodic & neumann (= 0) at top and bottom
			west {{ type = "inout"; rhs = 0.0; }}
			east {{ type = "inout"; rhs = 0.0; }}
			south {{ type = "inout"; rhs = 0.0; }}
			north {{ type = "inout"; rhs = 0.0; }}
		}}

		# --- sedimentation & deposition [optional, default = false]
		# 	--- require density & diameter setup
		# is_sedimentation = false;
		# is_dry_deposition = false;
		# is_wet_deposition = false;
		# density = 10.0 * 1000.0;	# [kg / m^3]
		# diameter = 10.0 * 0.000001;	# [m]
		
		# --- min limit [optional, default = none]
		# min_value = 0.0;
	}}
		"""
		
		# Add the text to initial line
		add_text_to_file(file_path, initial_line, text_to_add)
	change_lines(file_path, lines_to_change, lines_changed)
print(f"Text added successfully to {file_path} at line 346.")

Text added successfully to /app/nse/outputs/config-plume-ex_999.txt at line 346.


In [11]:
features_pd = pd.DataFrame({"y": y_list,
                            "z": z_list,
                            "u": u_list,
                            "power":p_list,
                            "roughness": z0_list,
                            "T": T0_list,
                            "sensible_heat_flux":value_list,
                            "T_grad":gradz_list})
features_pd.to_csv("features.csv", index=False)

In [ ]:
"""
LES Configuration Parser and Data Analyzer
----------------------------------------
A robust tool for parsing and analyzing LES configuration files with a focus on tracer data.

Author: Claude
Version: 1.0.0
"""

import re
from dataclasses import dataclass
from typing import List, Dict, Optional
import pandas as pd
import numpy as np
from pathlib import Path
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

@dataclass
class TracerData:
    """Data class for storing tracer-specific information."""
    index: int
    y_pos: float
    z_pos: float
    value: float

@dataclass
class ConfigData:
    """Data class for storing configuration parameters."""
    U: float
    z0_m: float
    surface_value: float
    grad_z: float
    tracers: List[TracerData]

class ConfigParser:
    """Parser for LES configuration files."""
    
    def __init__(self, config_path: str):
        """
        Initialize the parser with the path to the configuration file.
        
        Args:
            config_path (str): Path to the configuration file
        """
        self.config_path = Path(config_path)
        self.config_text = None
        self.config_data = None

    def read_config(self) -> None:
        """Read and store the configuration file content."""
        try:
            with open(self.config_path, 'r') as f:
                self.config_text = f.read()
            logger.info(f"Successfully read configuration file: {self.config_path}")
        except FileNotFoundError:
            logger.error(f"Configuration file not found: {self.config_path}")
            raise
        except Exception as e:
            logger.error(f"Error reading configuration file: {e}")
            raise

    def _extract_float_value(self, pattern: str) -> float:
        """
        Extract float value from config text using regex pattern.
        
        Args:
            pattern (str): Regex pattern to match
            
        Returns:
            float: Extracted value
        """
        match = re.search(pattern, self.config_text)
        if not match:
            raise ValueError(f"Could not find value matching pattern: {pattern}")
        return float(match.group(1))

    def _parse_tracers(self) -> List[TracerData]:
        """
        Parse tracer information from config text.
        
        Returns:
            List[TracerData]: List of tracer data objects
        """
        tracers = []
        tracer_sections = re.finditer(
            r'tracer_(\d+)\s*{[^}]*point_emission\s*{([^}]*)}', 
            self.config_text
        )
        
        for section in tracer_sections:
            index = int(section.group(1))
            tracer_text = section.group(2)
            
            try:
                value = float(re.search(r'value\s*=\s*([0-9.]+)', tracer_text).group(1))
                y_pos = float(re.search(r'ypos\s*=\s*[^+]*\+\s*([0-9.]+)', tracer_text).group(1))
                z_pos = float(re.search(r'zpos\s*=\s*([0-9.]+)', tracer_text).group(1))
                
                tracers.append(TracerData(index, y_pos, z_pos, value))
            except Exception as e:
                logger.error(f"Error parsing tracer {index}: {e}")
                raise
                
        return sorted(tracers, key=lambda x: x.index)

    def parse(self) -> ConfigData:
        """
        Parse the configuration file and extract relevant data.
        
        Returns:
            ConfigData: Parsed configuration data
        """
        if not self.config_text:
            self.read_config()

        try:
            # Extract main configuration values
            U = self._extract_float_value(r'U\s*=\s*([0-9.]+)')
            z0_m = self._extract_float_value(r'z0_m\s*=\s*([0-9.]+)')
            surface_value = self._extract_float_value(r'surface_value\s*=\s*([0-9.]+)')
            grad_z = self._extract_float_value(r'grad_z\s*=\s*([0-9.]+)')
            
            # Parse tracer data
            tracers = self._parse_tracers()
            
            self.config_data = ConfigData(U, z0_m, surface_value, grad_z, tracers)
            logger.info("Successfully parsed configuration data")
            
            return self.config_data
            
        except Exception as e:
            logger.error(f"Error parsing configuration: {e}")
            raise

class DataAnalyzer:
    """Analyzer for parsed configuration data."""
    
    def __init__(self, config_data: ConfigData):
        """
        Initialize the analyzer with parsed configuration data.
        
        Args:
            config_data (ConfigData): Parsed configuration data
        """
        self.config_data = config_data

    def create_dataframe(self) -> pd.DataFrame:
        """
        Create a pandas DataFrame from the configuration data.
        
        Returns:
            pd.DataFrame: DataFrame containing all configuration data
        """
        data = []
        
        # Create rows for each tracer
        for tracer in self.config_data.tracers:
            row = {
                'Tracer': tracer.index,
                'y (m)': tracer.y_pos,
                'z (m)': tracer.z_pos,
                'p (value)': tracer.value,
                'U (m/s)': self.config_data.U,
                'z0_m (m)': self.config_data.z0_m,
                'surface_value (K)': self.config_data.surface_value,
                'grad_z (K/m)': self.config_data.grad_z
            }
            data.append(row)
            
        return pd.DataFrame(data)

    def save_to_csv(self, output_path: str) -> None:
        """
        Save the data to a CSV file.
        
        Args:
            output_path (str): Path to save the CSV file
        """
        df = self.create_dataframe()
        df.to_csv(output_path, index=False)
        logger.info(f"Data saved to: {output_path}")

def main():
    """Main function to demonstrate usage."""
    try:
        # Initialize parser and parse config
        parser = ConfigParser("config.txt")
        config_data = parser.parse()
        
        # Analyze and save data
        analyzer = DataAnalyzer(config_data)
        df = analyzer.create_dataframe()
        
        # Display results
        print("\nParsed Configuration Data:")
        print("=" * 50)
        print(df.to_string(index=False))
        
        # Save to CSV
        analyzer.save_to_csv("les_config_analysis.csv")
        
    except Exception as e:
        logger.error(f"Error in main execution: {e}")
        raise

if __name__ == "__main__":
    main()